wip i need to finish
https://twitter.com/aicrumb


In [ ]:
%pip install -qq transformers accelerate bitsandbytes optimum

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm, trange
import torch

# imagine these are better models
big_model_id = "gpt2-xl"
small_model_id = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(big_model_id) # they should use the same tokenizer

big_model = AutoModelForCausalLM.from_pretrained(big_model_id, load_in_4bit=True, device_map="auto")
small_model = AutoModelForCausalLM.from_pretrained(small_model_id, device_map="auto")

big_model = big_model.to_bettertransformer()
small_model = small_model.to_bettertransformer()

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.
The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [ ]:
# adjust batch size for your gpu, sequence length, and model

prompt = "ago in a galaxy far, far away..." # target: A long time ago in a galaxy far, far away...
batch_size = 512
prompt_tokens = tokenizer.encode(prompt, return_tensors='pt')
possible_tokens = list(range(tokenizer.vocab_size))

def generator(x, batch_size):
    for i in range(0, len(x) // batch_size + 1):
        yield(x[i*batch_size:i*batch_size+batch_size])

tokens = list(generator(possible_tokens, batch_size))

In [ ]:
loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
perplexities = []
for batch_pre_tokens in tqdm(tokens):
    batch_prompt_tokens = prompt_tokens.repeat(len(batch_pre_tokens), 1)
    batch_prompt_tokens = torch.cat([torch.tensor(batch_pre_tokens).unsqueeze(-1), batch_prompt_tokens], -1)
    batch_prompt_tokens = batch_prompt_tokens.cuda()
    with torch.no_grad():
        # just the forward method of gpt2 basically copy/pasted without extra fluff
        # https://github.com/huggingface/transformers/blob/d1a00f9dd0b851245e4a54cbd70816a80e781ec2/src/transformers/models/gpt2/modeling_gpt2.py#L1049
        transformer_outputs = small_model.transformer(batch_prompt_tokens)
        hidden_states = transformer_outputs[0]
        lm_logits = small_model.lm_head(hidden_states)
        shift_logits = lm_logits[..., :-1, :].contiguous()
        shift_labels = batch_prompt_tokens[..., 1:].contiguous()
        # deviate here :/
        shift_labels = torch.nn.functional.one_hot(shift_labels, tokenizer.vocab_size)
        loss = loss_fct(shift_logits, shift_labels.float())
        loss = loss.tolist()
        perplexities += loss


  0%|          | 0/99 [00:00<?, ?it/s]

In [ ]:
torch.topk(torch.tensor(perplexities), 8)

In [ ]:
# okay if this didn't use up all of the ram then i would say "then you choose the top k (like, maybe 20) indices and either
# sample from them the same way we sample for transformers or then run them through the bigger model
# to find a smaller top k to sample from or just the top most probable"